# Data Preprocessing

In [1]:
import numpy as np
# Given data matrix, (-1,-1) represents blockage , (100,100) represents Goal State
A= np.array([
    [(0.2,0.9),(0.8,1),(0.05,0.05),(0.8,0.9),(0.8,0.9),(-1,-1),(100,100),(0.05,0.05),(0.2,0.1),(-1,-1)],
    [(-1,-1),(0.05,0.05),(0.05,0.05),(-1,-1),(0.8,0.9),(0.8,0.9),(0.05,0.05),(0.05,0.05),(0.2,0.1),(0.2,0.1)],
    [(0.2,0.9),(-1,-1),(0.2,0.1),(-1,-1),(0.2,0.1),(-1,-1),(-1,-1),(-1,-1),(0.2,0.1),(0.2,0.1)],
    [(0.2,0.9),(-1,-1),(0.2,0.1),(-1,-1),(0.2,0.1),(0.3,0.9),(0.2,0.1),(0.2,0.1),(0.2,0.1),(0.2,0.1)],
    [(0.2,0.9),(-1,-1),(0.2,0.1),(-1,-1),(0.3,0.9),(0.2,0.1),(0.2,0.1),(0.2,0.9),(0.2,0.9),(0.2,0.1)],
    [(0.2,0.9),(0.2,1),(0.2,0.1),(0.3,0.9),(0.3,0.9),(0.2,0.1),(-1,-1),(-1,-1),(-1,-1),(0.05,0.05)],
    [(0.2,0),(0.2,0.1),(0.2,0.1),(0.05,0.05),(0.05,0.05),(0.05,0.05),(-1,-1),(0.05,0.05),(0.05,0.05),(0.05,0.05)],
    [(0,0),(0,0),(0,0),(0.05,0.05),(0.05,0.05),(0.05,0.05),(-1,-1),(0.05,0.05),(0.05,0.05),(0.05,0.05)],
    [(0,0),(0,0),(0,0),(0.05,0.05),(0.05,0.05),(-1,-1),(0.05,0.05),(0.05,0.05),(-1,-1),(0.05,0.05)],
    [(0,0),(0,0),(0,0),(-1,-1),(-1,-1),(-1,-1),(0.05,0.05),(0.05,0.05),(-1,-1),(-1,-1)]
    ])  
# Maze matrix values (x,y) whereas x is probability of mine and y is the probability of detection (PoD).

In [2]:
#Calculate [(1+PoD)(1+PoM)] for each cell with value PoD/PoM, whereas -1 represents blockage and Goal represent Goat state
final=[]
for i in A:
    inter=[]
    for j in i:
        if j[0]==100:
            inter.append('Goal') #Goal State
        elif j[0]!=-1:
            inter.append((j[0]+1)*(j[1]+1))#Calculate fitness value for each coordinates and replace (x,y) with [(1+PoD)(1+PoM)]
        else:
            inter.append('B')  #No Path
    final.append(inter)

print('The Matrix is: \n',final)

The Matrix is: 
 [[2.28, 3.6, 1.1025, 3.42, 3.42, 'B', 'Goal', 1.1025, 1.32, 'B'], ['B', 1.1025, 1.1025, 'B', 3.42, 3.42, 1.1025, 1.1025, 1.32, 1.32], [2.28, 'B', 1.32, 'B', 1.32, 'B', 'B', 'B', 1.32, 1.32], [2.28, 'B', 1.32, 'B', 1.32, 2.4699999999999998, 1.32, 1.32, 1.32, 1.32], [2.28, 'B', 1.32, 'B', 2.4699999999999998, 1.32, 1.32, 2.28, 2.28, 1.32], [2.28, 2.4, 1.32, 2.4699999999999998, 2.4699999999999998, 1.32, 'B', 'B', 'B', 1.1025], [1.2, 1.32, 1.32, 1.1025, 1.1025, 1.1025, 'B', 1.1025, 1.1025, 1.1025], [1.0, 1.0, 1.0, 1.1025, 1.1025, 1.1025, 'B', 1.1025, 1.1025, 1.1025], [1.0, 1.0, 1.0, 1.1025, 1.1025, 'B', 1.1025, 1.1025, 'B', 1.1025], [1.0, 1.0, 1.0, 'B', 'B', 'B', 1.1025, 1.1025, 'B', 'B']]


# Random Restart Hill Climbing algorithm

In [6]:
import random
def goal_test(neighbores, neighbores_coords): # Check the Goal condition if reached goal or not 
    if 'Goal' in neighbores:
        return neighbores_coords[neighbores.index('Goal')]  #Return index of Goal
    return False # return false if goal not found

def find_neighbore(x, y, path):
    if (x == 0 and y == 0):   # Top-Left Bounary conditions
        neighbores = [path[x][y+1]]
        neighbores_coords = [(x,y+1)]
    elif x == 0: # Top Bounary conditions
        neighbores = [path[x][y+1],path[x+1][y]]
        neighbores_coords = [(x,y+1),(x+1,y)]
    elif (x == 0 and y == len(path)-1): # Top-Right Bounary conditions
        neighbores = [path[x-1][y]]
        neighbores_coords = [(x-1,y)]
    elif (x == len(path)-1 and y == len(path)-1) or y == len(path)-1: # Right-bottom Bounary conditions
        neighbores = [path[x][y-1], path[x-1][y]]
        neighbores_coords = [(x,y-1),(x-1,y)]
    elif (x == len(path)-1 and y == 0) or y == 0:   # Left-bottom Bounary conditions
        neighbores = [path[x][y+1], path[x-1][y]]
        neighbores_coords = [(x,y+1),(x-1,y)]
    else: # Ideal Case
        neighbores = [path[x][y+1], path[x][y-1], path[x-1][y]]
        neighbores_coords = [(x,y+1),(x,y-1),(x-1,y)]

    return neighbores, neighbores_coords  # return the boundaries of the current nodes

def best_neighbore(path, starting_point, path_value): #Find the next best value of the current position
    x , y = starting_point[0] , starting_point[1]
    neighbores, neighbores_coords = find_neighbore(x, y, path) #find all neighbore of the current nodes
    IsGoal = goal_test(neighbores, neighbores_coords)  #Check if Goal state meets
    if IsGoal != False:  
        return True, IsGoal, 0  #Return coordinates if Goal node found
        
    neighbore_min = neighbores
    if 'B' in neighbore_min:  # Remove noPath/Blockage neighbore
        neighbore_min = [i for i in neighbore_min if i != 'B']
    for i in range(len(neighbore_min)):
        best_cost_value = sorted(neighbore_min)[i]  #Minimize(f(n))
        
        if neighbore_min.count(best_cost_value)>1:
            #find best coordinates if repeated values
            best_neighbore_value = neighbores_coords[neighbores.index(best_cost_value)+i]  
        else:
            #find best coordinates if values are not repeated
            best_neighbore_value = neighbores_coords[neighbores.index(best_cost_value)] 
        if best_neighbore_value not in path_value: #If 
            break
    if best_neighbore_value in path_value and i == len(neighbore_min)-1: #If Agent blocked then return None
        best_neighbore_value = None
    
    return IsGoal, best_neighbore_value, best_cost_value

def random_restart_hill_climbing(path):
    starting_point_list = [(9,0),(8,0),(7,0),(9,1),(9,2)] 
    starting_point = starting_point_list[random.randrange(len(starting_point_list))]#Randomly select starting node from the maze
    #print('Starting Point: ',starting_point)
    cost_value = []
    path_value = []
    path_value.append(starting_point)
    for i in range(50):
        #find next best neighbore value or next best move and associates cost
        Goal_check, best_neighbore_value, best_cost_value = best_neighbore(path, starting_point, path_value) 
        cost_value.append(best_cost_value)
        path_value.append(best_neighbore_value)
        if Goal_check == True or best_neighbore_value == None: #Loop break if Goal Found
            break 
        starting_point = best_neighbore_value  #Update current position of the agent
        
    #print('The path coodinates are : ', path_value)
    #print('The path cost is :', sum(cost_value))
    return path_value, sum(cost_value)


In [16]:
import time 

def random_restart_HC(problem, limit=10):
    start = time.time()
    count = 0
    optimal_path = []
    optimal_cost = []
    while count < limit:
        path_value, cost_value = random_restart_hill_climbing(problem)
        optimal_path.append(path_value)
        optimal_cost.append(float(cost_value))
        count += 1
    end = time.time()
    print('The optimal Path: ', optimal_path[optimal_cost.index(min(optimal_cost))])
    print('The optimal Cost: ', min(optimal_cost))
    print('The Space and Time Complexity is ', end - start)    
    
random_restart_HC(final, 5)
 

The optimal Path:  [(9, 1), (9, 2), (8, 2), (8, 1), (8, 0), (7, 0), (7, 1), (7, 2), (7, 3), (6, 3), (6, 4), (6, 5), (5, 5), (4, 5), (4, 6), (3, 6), (3, 7), (3, 8), (3, 9), (2, 9), (2, 8), (2, 7), (1, 7), (1, 6), (0, 6)]
The optimal Cost:  26.814999999999998
The Space and Time Complexity is  0.004638671875
